In [1]:
import os
import cv2
import numpy as np

In [2]:
def xywh2xyxy(size, box):
        """
        Param:  self
                +size: (Tuple)
                +box: (Numpy array)
 
        Using:  +size: (Tuple)
                +box: (Numpy array)
 
        Does:   Gets a numpy xywh Yolo format
                array and transforms to x1y1x2y2  format.
        
        Return: (x1, y1, x2, y2)
        """
        width, height = size
        x_center, y_center, w, h = box
        x1 = (x_center - w / 2) * width
        y1 = (y_center - h / 2) * height
        x2 = (x_center + w / 2) * width
        y2 = (y_center + h / 2) * height
        return (x1, y1, x2, y2)

In [3]:
def xyxyx2xywh(size, box):
        """
        Param:  self
                +size: (Tuple)
                +box: (Numpy array)

        Using:  +size: (Tuple)
                +box: (Numpy array)

        Does:   Gets a numpy x1y1x2y2 array and
                transforms to xywh YOLO format.
        
        Return: x, y, w, h
        """
        img_width, img_height = size
        x1, y1, x2, y2 = box

        x_center = (x1 + x2) / 2.0
        y_center = (y1 + y2) / 2.0
        
        width = x2 - x1
        height = y2 - y1
        
        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height
        
        return x_center, y_center, width, height

In [12]:
path = '/home/bdogan/Desktop/obj_train_data/'
temp_destination = '/home/bdogan/Desktop/test/'
ratio = 1.15

files = os.listdir(path)

for file in files:
    if file.endswith('.jpg'):

        temp = file.split('.jpg')

        first_name = temp[0]

        img = cv2.imread(path + first_name + '.jpg')
        height, width, channels = img.shape

        reading_file = open(path + first_name + '.txt',"r")
        new_str = ''

        for line in reading_file:
            
            words = (line.split())
            size = (width, height)
            box = [float(words[1]), float(words[2]), float(words[3]), float(words[4])]

            x1, y1, x2, y2 = xywh2xyxy(size, np.array(box))

            if y1 - ((y2 - y1) * ratio) > 0 and y2 + ((y2 - y1) * ratio) < height and x1 - ((x2 - x1) * ratio) > 0 and x2 + ((x2 - x1) * ratio) < width:
                temp_x1 = x1 - ((ratio - 1) * x1)
                temp_y1 = y1 - ((ratio - 1) * y1)
                temp_x2 = x2 + ((ratio - 1) * x2)
                temp_y2 = y2 + ((ratio - 1) * y2)

                temp_box = [temp_x1, temp_y1, temp_x2, temp_y2]

                new_x1, new_y1, new_x2, new_y2 = xyxyx2xywh(size=size, box=temp_box)

                new_str += f'{words[0]} {new_x1} {new_y1} {new_x2} {new_y2}\n'
            
            else:
                print(first_name)
                os.remove(path + first_name + '.jpg')
                os.remove(path + first_name + '.txt')
                break

        writing_file = open(temp_destination + first_name + '.txt',"w")
        writing_file.write(new_str)
        writing_file.close()
        
        
        cropped_image = img[int(temp_y1) : int(temp_y2), int(temp_x1) : int(temp_x2)]
        cv2.imwrite(temp_destination + first_name + '.jpg', cropped_image)

38d66bee_frame532
38d66bee_frame5381
38d66bee_frame5432
38d66bee_frame5281
38d66bee_frame113
38d66bee_frame5330
38d66bee_frame4664
38d66bee_frame3779
38d66bee_frame3925
38d66bee_frame2690
38d66bee_frame1535
38d66bee_frame3780
38d66bee_frame2018
38d66bee_frame3782
38d66bee_frame1318
38d66bee_frame1835
38d66bee_frame3294
38d66bee_frame5415
38d66bee_frame1749
38d66bee_frame3781
38d66bee_frame34
38d66bee_frame3121
38d66bee_frame3783
38d66bee_frame2266
38d66bee_frame1833
38d66bee_frame1834
38d66bee_frame112
38d66bee_frame3122
38d66bee_frame4662
38d66bee_frame3041
38d66bee_frame796
38d66bee_frame3786
38d66bee_frame346
38d66bee_frame3784
38d66bee_frame5414
38d66bee_frame3785
38d66bee_frame5231
38d66bee_frame3968
38d66bee_frame1319
38d66bee_frame93
38d66bee_frame5077
38d66bee_frame2504
38d66bee_frame3120
38d66bee_frame1322
38d66bee_frame3242


In [ ]:
import os
import cv2

def crop_and_resize_images(txt_folder, image_dir, output_dir):
    for txt_file_name in os.listdir(txt_folder):
        count = 0
        if txt_file_name.endswith('.txt'):
            txt_file_path = os.path.join(txt_folder, txt_file_name)
            with open(txt_file_path, 'r') as file:
                lines = file.readlines()

            for line in lines:
                parts = line.strip().split()
                if len(parts) < 5:
                    continue
                
                class_id = int(parts[0])
                x, y, width, height = map(float, parts[1:])

                image_name = os.path.splitext(txt_file_name)[0] + '.jpg'
                image_path = os.path.join(image_dir, image_name)
                
                if not os.path.exists(image_path):
                    print(f"Image {image_name} not found.")
                    continue
                
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Failed to load image: {image_path}")
                    continue
                
                image_height, image_width = image.shape[:2]
                
                x1 = int((x - width / 2) * image_width)
                y1 = int((y - height / 2) * image_height)
                x2 = int((x + width / 2) * image_width)
                y2 = int((y + height / 2) * image_height)

                x1 = max(0, min(x1, image_width - 1))
                y1 = max(0, min(y1, image_height - 1))
                x2 = max(0, min(x2, image_width - 1))
                y2 = max(0, min(y2, image_height - 1))

                padding_x = int((x2 - x1) * 0.7)
                padding_y = int((y2 - y1) * 3.0)

                x1 = max(0, x1 - padding_x)
                y1 = max(0, y1 - padding_y)
                x2 = min(image_width, x2 + padding_x)
                y2 = min(image_height, y2 + padding_y)

                cropped_image = image[y1:y2, x1:x2]

                new_image_name = os.path.splitext(image_name)[0] + f'_{count}_cropped.jpg'
                new_image_path = os.path.join(output_dir, new_image_name)
                cv2.imwrite(new_image_path, cropped_image)
                count += 1

                new_x = (x - (x1 / image_width)) / ((x2 - x1) / image_width)
                new_y = (y - (y1 / image_height)) / ((y2 - y1) / image_height)
                new_width = width / ((x2 - x1) / image_width)
                new_height = height / ((y2 - y1) / image_height)


                new_txt_name = os.path.splitext(new_image_name)[0] + '.txt'
                new_txt_path = os.path.join(output_dir, new_txt_name)
                with open(new_txt_path, 'a') as new_txt_file:
                    new_txt_file.write(f"{class_id} {new_x} {new_y} {new_width} {new_height}\n")

def main():
    txt_folder = "/home/bdogan/Downloads/deneme"  # Path to the folder containing text files with bounding box annotations
    image_dir = txt_folder  # Directory containing the images
    output_dir = "/home/bdogan/Downloads/temp/"  # Output directory for cropped images and annotations

    crop_and_resize_images(txt_folder, image_dir, output_dir)

    print("Image cropping and annotation updating completed.")

if __name__ == "__main__":
    main()

Image cropping and annotation updating completed.


# Crop image and move annotation to it's new place

In [ ]:
import os
import cv2

def crop_image_and_update_labels(image_path, label_path, x1, y1, x2, y2, output_image_path, output_label_path):
    image = cv2.imread(image_path)
    h, w, _ = image.shape

    cropped_image = image[y1:y2, x1:x2]
    cv2.imwrite(output_image_path, cropped_image)

    with open(label_path, 'r') as f:
        labels = f.readlines()

    new_labels = []
    for label in labels:
        parts = label.strip().split()
        class_id = parts[0]
        bbox_x = float(parts[1]) * w
        bbox_y = float(parts[2]) * h
        bbox_w = float(parts[3]) * w
        bbox_h = float(parts[4]) * h

        new_x = (bbox_x - x1) / (x2 - x1)
        new_y = (bbox_y - y1) / (y2 - y1)
        new_w = bbox_w / (x2 - x1)
        new_h = bbox_h / (y2 - y1)

        if 0 <= new_x <= 1 and 0 <= new_y <= 1:
            new_labels.append(f"{class_id} {new_x} {new_y} {new_w} {new_h}\n")

    with open(output_label_path, 'w') as f:
        f.writelines(new_labels)

def process_folder(input_folder, output_folder, x1, y1, x2, y2):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith('.jpg'):
            image_path = os.path.join(input_folder, filename)
            label_path = os.path.join(input_folder, filename.replace('.jpg', '.txt'))
            output_image_path = os.path.join(output_folder, filename)
            output_label_path = os.path.join(output_folder, filename.replace('.jpg', '.txt'))

            if os.path.exists(label_path):
                crop_image_and_update_labels(image_path, label_path, x1, y1, x2, y2, output_image_path, output_label_path)

input_folder = '/home/bdogan/all_files/ekin_files/tbatuhanl/traffic_light_detection/cvat_sets_saha/000_ayrı_saha_3/temp'
output_folder = '/home/bdogan/all_files/ekin_files/tbatuhanl/traffic_light_detection/cvat_sets_saha/000_ayrı_saha_3/1'
x1, y1, x2, y2 = 677, 581, 791, 808

process_folder(input_folder, output_folder, x1, y1, x2, y2)
